# **Importación de librerías y carga de datos**

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import json
from collections import Counter
import unicodedata
from fuzzywuzzy import fuzz, process

import warnings
warnings.filterwarnings('ignore')

# Para procesamiento de texto
import unicodedata
from difflib import SequenceMatcher
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz, process

# Carga de datos
print("Cargando datos...")
df_mensajes = pd.read_csv('mensajes.csv')
print(f"Total de mensajes cargados: {len(df_mensajes)}")
print(f"Columnas: {df_mensajes.columns.tolist()}")
print(df_mensajes.head())

/usr/local/lib/python3.12/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Cargando datos...


FileNotFoundError: [Errno 2] No such file or directory: 'mensajes.csv'

# **Limpieza de texto**

In [ ]:
def limpiar_texto(texto):
    """
    Limpia el texto eliminando emojis, URLs y caracteres especiales
    """
    if pd.isna(texto):
        return ""

    # Convertir a string
    texto = str(texto)

    # Eliminar URLs
    texto = re.sub(r'http\S+|www.\S+', '', texto)

    # Eliminar emojis y caracteres especiales
    texto = re.sub(r'[^\w\s\,\.\:\;\-\$\%\/\(\)]', ' ', texto)

    # Eliminar múltiples espacios
    texto = re.sub(r'\s+', ' ', texto)

    # Eliminar espacios al inicio y final
    texto = texto.strip()

    return texto

def eliminar_duplicados(df):
    """
    Elimina mensajes duplicados basándose en el contenido
    """
    # Crear columna temporal con texto limpio para comparación
    df['contenido_limpio_temp'] = df['contenido'].apply(limpiar_texto)

    # Eliminar duplicados exactos
    df_sin_dup = df.drop_duplicates(subset=['contenido_limpio_temp'], keep='first')

    # Eliminar la columna temporal
    df_sin_dup = df_sin_dup.drop('contenido_limpio_temp', axis=1)

    print(f"Mensajes originales: {len(df)}")
    print(f"Mensajes después de eliminar duplicados: {len(df_sin_dup)}")

    return df_sin_dup

# Aplicar limpieza
print("\nLimpiando textos...")
df_mensajes['contenido_limpio'] = df_mensajes['contenido'].apply(limpiar_texto)

print("\nEliminando duplicados...")
df_limpio = eliminar_duplicados(df_mensajes)

# Filtrar mensajes con contenido relevante (más de 50 caracteres)
df_limpio = df_limpio[df_limpio['contenido_limpio'].str.len() > 50].copy()
print(f"Mensajes con contenido relevante: {len(df_limpio)}")


Limpiando textos...


NameError: name 'df_mensajes' is not defined

# **Extracción de información**

In [ ]:
def extraer_informacion_oferta(texto):
    """
    Extrae información estructurada de la oferta laboral
    """
    info = {
        'cargo': None,
        'empresa': None,
        'ubicacion': None,
        'salario': None,
        'tipo_contrato': None,
        'nivel_educativo': None,
        'experiencia': None,
        'vacantes': None,
        'fecha_evento': None,
        'lugar_evento': None
    }

    # Extraer cargo (buscar después de palabras clave)
    cargo_patterns = [
        r'vacante[s]?\s+(?:para\s+)?([^\n\r\.]+)',
        r'busca[n]?\s+(\d+)?\s*([^\n\r\.]+?)(?:\s+con|\s+Tipo|\s+Nivel|\s+Experiencia)',
        r'puestos?\s+de\s+trabajo\s+(?:para|como)?\s*([^\n\r\.]+)',
        r'cargo[s]?:?\s*([^\n\r\.]+)',
    ]

    for pattern in cargo_patterns:
        match = re.search(pattern, texto, re.IGNORECASE)
        if match:
            if match.lastindex >= 2:
                info['cargo'] = match.group(2).strip()
            else:
                info['cargo'] = match.group(1).strip()
            break

    # Extraer empresa
    empresa_patterns = [
        r'([A-Z][a-zA-Z\s]+(?:S\.A\.S|SAS|S\.A|SA|Ltda|LTDA))',
        r'(?:empresa|compañía):\s*([^\n\r\.]+)',
    ]

    for pattern in empresa_patterns:
        match = re.search(pattern, texto)
        if match:
            info['empresa'] = match.group(1).strip()
            break

    # Extraer ubicación/ciudad
    ciudades = ['Bogotá', 'Medellín', 'Cali', 'Barranquilla', 'Cartagena',
                'Suba', 'Kennedy', 'Fontibón', 'Chapinero', 'Bosa', 'Usaquén']

    for ciudad in ciudades:
        if ciudad.lower() in texto.lower():
            info['ubicacion'] = ciudad
            break

    # Extraer salario
    salario_match = re.search(r'\$\s*[\d\.,]+', texto)
    if salario_match:
        info['salario'] = salario_match.group(0).strip()

    # Extraer tipo de contrato
    contratos = ['indefinido', 'término fijo', 'termino fijo', 'obra labor', 'prestación de servicios']
    for contrato in contratos:
        if contrato in texto.lower():
            info['tipo_contrato'] = contrato.title()
            break

    # Extraer nivel educativo
    niveles = ['bachiller', 'técnico', 'tecnólogo', 'profesional', 'especialización', 'maestría']
    for nivel in niveles:
        if nivel in texto.lower():
            info['nivel_educativo'] = nivel.title()
            break

    # Extraer experiencia
    exp_match = re.search(r'(\d+)\s*(?:mes|año)[s]?\s+(?:de\s+)?experiencia', texto, re.IGNORECASE)
    if exp_match:
        info['experiencia'] = exp_match.group(0).strip()
    elif 'sin experiencia' in texto.lower() or 'no requiere experiencia' in texto.lower():
        info['experiencia'] = 'No requiere'

    # Extraer número de vacantes
    vacantes_match = re.search(r'(\d+)\s+(?:puestos?|vacantes?)', texto, re.IGNORECASE)
    if vacantes_match:
        info['vacantes'] = int(vacantes_match.group(1))

    # Extraer fecha de evento
    fecha_match = re.search(r'(\d{1,2})\s+de\s+(\w+)', texto, re.IGNORECASE)
    if fecha_match:
        info['fecha_evento'] = fecha_match.group(0).strip()

    # Extraer lugar del evento
    lugar_patterns = [
        r'(?:Lugar|Dónde|Donde):\s*([^\n\r]+?)(?:\n|\r|Fecha|Hora|$)',
        r'(?:Centro Comercial|C\.C\.|Calle|Carrera|Av\.|Avenida)\s+([^\n\r]+?)(?:\n|\r|Fecha|$)'
    ]

    for pattern in lugar_patterns:
        match = re.search(pattern, texto, re.IGNORECASE)
        if match:
            info['lugar_evento'] = match.group(1).strip()[:100]
            break

    return info

# Aplicar extracción
print("\nExtrayendo información de ofertas...")
info_extraida = df_limpio['contenido_limpio'].apply(extraer_informacion_oferta)
df_info = pd.DataFrame(info_extraida.tolist())

# Combinar con el dataframe original
df_procesado = pd.concat([df_limpio.reset_index(drop=True), df_info], axis=1)

print("\nPrimeras extracciones:")
print(df_procesado[['cargo', 'empresa', 'ubicacion', 'nivel_educativo']].head(10))

# **Carga y preparación de CUOC**

In [ ]:
import pandas as pd
import re

print("\nCargando bases CUOC...")

# =========================================================
# 1️⃣ ESTRUCTURA CUOC (una sola columna con código y descripción)
# =========================================================
try:
    df_cuoc_estructura = pd.read_excel(
        "CUOC-Estructura-2024.xlsx",
        header=3,     # salta logo y encabezado visual
        usecols="A",  # solo hay una columna
        names=["texto"]
    )

    # Extraer código y descripción (p.ej. "0110 Oficiales de las Fuerzas Militares")
    df_cuoc_estructura[["codigo_cuoc", "ocupacion"]] = (
        df_cuoc_estructura["texto"]
        .astype(str)
        .str.extract(r"^(\d+[A-Za-z0-9\.]*)\s+(.*)$")
    )

    # Limpiar filas sin código o sin ocupación
    df_cuoc_estructura = df_cuoc_estructura.dropna(subset=["codigo_cuoc", "ocupacion"])
    df_cuoc_estructura = df_cuoc_estructura.reset_index(drop=True)

    print(f"Estructura CUOC cargada: {len(df_cuoc_estructura)} registros")
    print(f"Columnas: {df_cuoc_estructura.columns.tolist()}")

except Exception as e:
    print(f"❌ No se pudo cargar CUOC-Estructura-2024.xlsx: {e}")
    df_cuoc_estructura = None


# =========================================================
# 2️⃣ ÍNDICE CUOC (tiene 2 columnas: código y descripción)
# =========================================================
try:
    df_cuoc_indice = pd.read_excel(
        "CUOC-indice-2024.xlsx",
        header=5,  # después del título "ÍNDICE CUOC 2024"
        usecols="B:C"
    )
    df_cuoc_indice.columns = ["codigo_cuoc", "ocupacion"]
    df_cuoc_indice = df_cuoc_indice.dropna(subset=["codigo_cuoc", "ocupacion"])
    df_cuoc_indice = df_cuoc_indice.reset_index(drop=True)

    print(f"Índice CUOC cargado: {len(df_cuoc_indice)} registros")
    print(f"Columnas: {df_cuoc_indice.columns.tolist()}")

except Exception as e:
    print(f"❌ No se pudo cargar CUOC-indice-2024.xlsx: {e}")
    df_cuoc_indice = None


# **Asignación de código CUOC mediante similitud**

In [ ]:

def normalizar_texto_cuoc(texto):
    """
    Normaliza texto para comparación CUOC
    """
    if pd.isna(texto):
        return ""

    texto = str(texto).lower()
    # Eliminar acentos
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto)
                    if unicodedata.category(c) != 'Mn')
    # Eliminar caracteres especiales
    texto = re.sub(r'[^a-z0-9\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto


def buscar_codigo_cuoc(cargo, df_cuoc, umbral=60):
    """
    Busca el código CUOC más similar al cargo usando fuzzy matching
    """
    if pd.isna(cargo) or cargo == "" or df_cuoc is None or len(df_cuoc) == 0:
        return {
            'codigo_cuoc': None,
            'ocupacion_cuoc': None,
            'similitud': 0,
            'grupo_cuoc': None
        }

    cargo_norm = normalizar_texto_cuoc(cargo)

    # Crear lista de ocupaciones normalizadas
    ocupaciones = df_cuoc['ocupacion'].fillna('').tolist()
    ocupaciones_norm = [normalizar_texto_cuoc(o) for o in ocupaciones]

    # Buscar la mejor coincidencia
    mejor_match = process.extractOne(
        cargo_norm,
        ocupaciones_norm,
        scorer=fuzz.token_sort_ratio
    )

    # Si encontró una coincidencia válida
    if mejor_match and len(mejor_match) >= 2 and mejor_match[1] >= umbral:
        texto_match = mejor_match[0]
        similitud = mejor_match[1]

        # Buscar el índice de esa coincidencia
        try:
            idx = ocupaciones_norm.index(texto_match)
        except ValueError:
            idx = None

        if idx is not None:
            return {
                'codigo_cuoc': df_cuoc.iloc[idx]['codigo_cuoc'],
                'ocupacion_cuoc': df_cuoc.iloc[idx]['ocupacion'],
                'similitud': similitud,
                'grupo_cuoc': df_cuoc.iloc[idx].get('grupo', None)
            }

    # Si no hay coincidencia suficiente
    return {
        'codigo_cuoc': None,
        'ocupacion_cuoc': None,
        'similitud': 0,
        'grupo_cuoc': None
    }


# ============================================
# Asignar códigos CUOC
# ============================================

print("\nAsignando códigos CUOC a las ofertas...")

if df_cuoc_indice is not None:
    # Normalizar columna de ocupaciones en CUOC
    df_cuoc_indice['ocupacion_norm'] = df_cuoc_indice['ocupacion'].apply(normalizar_texto_cuoc)

    # Aplicar búsqueda a cada cargo
    cuoc_asignado = df_procesado['cargo'].apply(
        lambda x: buscar_codigo_cuoc(x, df_cuoc_indice, umbral=60)
    )

    # Convertir a DataFrame
    df_cuoc_result = pd.DataFrame(cuoc_asignado.tolist())

    # Combinar con el dataframe procesado
    df_final = pd.concat([df_procesado.reset_index(drop=True), df_cuoc_result], axis=1)

    print("\nAsignaciones CUOC exitosas:")
    print(df_final[df_final['codigo_cuoc'].notna()][
        ['cargo', 'ocupacion_cuoc', 'codigo_cuoc', 'similitud']
    ].head(10))

    print(f"\nTotal de ofertas con código CUOC: {df_final['codigo_cuoc'].notna().sum()}")
    print(f"Porcentaje de asignación: {(df_final['codigo_cuoc'].notna().sum()/len(df_final)*100):.2f}%")
else:
    df_final = df_procesado.copy()
    print("No se pudo realizar asignación CUOC (falta archivo)")



# **Análisis descriptivo**

In [ ]:
def analisis_descriptivo(df):
    """
    Genera análisis descriptivo completo de las ofertas
    """
    analisis = {
        'resumen_general': {},
        'ocupaciones_frecuentes': {},
        'grupos_ocupacionales': {},
        'ubicaciones': {},
        'nivel_educativo': {},
        'experiencia': {},
        'salarios': {},
        'tipo_contrato': {},
        'empresas_top': {},
        'tendencia_temporal': {}
    }

    # RESUMEN GENERAL
    analisis['resumen_general'] = {
        'total_ofertas': len(df),
        'ofertas_con_cargo_identificado': df['cargo'].notna().sum(),
        'ofertas_con_cuoc': df['codigo_cuoc'].notna().sum() if 'codigo_cuoc' in df.columns else 0,
        'total_vacantes': df['vacantes'].sum() if 'vacantes' in df.columns else 0,
        'empresas_unicas': df['empresa'].nunique() if 'empresa' in df.columns else 0
    }

    # OCUPACIONES MÁS FRECUENTES
    if 'cargo' in df.columns:
        top_cargos = df['cargo'].value_counts().head(20)
        analisis['ocupaciones_frecuentes'] = {
            str(k): int(v) for k, v in top_cargos.items()
        }

    # GRUPOS OCUPACIONALES (CUOC)
    if 'grupo_cuoc' in df.columns:
        grupos = df['grupo_cuoc'].value_counts()
        analisis['grupos_ocupacionales'] = {
            str(k): int(v) for k, v in grupos.items()
        }

    # UBICACIONES
    if 'ubicacion' in df.columns:
        ubicaciones = df['ubicacion'].value_counts()
        analisis['ubicaciones'] = {
            str(k): int(v) for k, v in ubicaciones.items()
        }

    # NIVEL EDUCATIVO
    if 'nivel_educativo' in df.columns:
        niveles = df['nivel_educativo'].value_counts()
        analisis['nivel_educativo'] = {
            str(k): int(v) for k, v in niveles.items()
        }

    # EXPERIENCIA
    if 'experiencia' in df.columns:
        experiencia = df['experiencia'].value_counts()
        analisis['experiencia'] = {
            str(k): int(v) for k, v in experiencia.items()
        }

    # TIPO DE CONTRATO
    if 'tipo_contrato' in df.columns:
        contratos = df['tipo_contrato'].value_counts()
        analisis['tipo_contrato'] = {
            str(k): int(v) for k, v in contratos.items()
        }

    # EMPRESAS TOP
    if 'empresa' in df.columns:
        top_empresas = df['empresa'].value_counts().head(15)
        analisis['empresas_top'] = {
            str(k): int(v) for k, v in top_empresas.items()
        }

    # TENDENCIA TEMPORAL
    if 'fecha_hora' in df.columns:
        df['fecha'] = pd.to_datetime(df['fecha_hora']).dt.date
        temporal = df.groupby('fecha').size()
        analisis['tendencia_temporal'] = {
            str(k): int(v) for k, v in temporal.items()
        }

    return analisis

# Ejecutar análisis
print("\nGenerando análisis descriptivo...")
resultados_analisis = analisis_descriptivo(df_final)

# Mostrar resumen
print("\n=== RESUMEN GENERAL ===")
for key, value in resultados_analisis['resumen_general'].items():
    print(f"{key}: {value}")

print("\n=== TOP 10 OCUPACIONES ===")
for i, (cargo, freq) in enumerate(list(resultados_analisis['ocupaciones_frecuentes'].items())[:10], 1):
    print(f"{i}. {cargo}: {freq} ofertas")

print("\n=== DISTRIBUCIÓN POR UBICACIÓN ===")
for ciudad, count in resultados_analisis['ubicaciones'].items():
    print(f"{ciudad}: {count} ofertas")

print("\n=== NIVEL EDUCATIVO ===")
for nivel, count in resultados_analisis['nivel_educativo'].items():
    print(f"{nivel}: {count} ofertas")

# **Análisis de salarios**

In [ ]:
def extraer_valor_salario(salario_texto):
    """
    Extrae el valor numérico del salario
    """
    if pd.isna(salario_texto):
        return None

    # Eliminar símbolos y convertir a número
    numeros = re.findall(r'\d+', str(salario_texto).replace(',', '').replace('.', ''))

    if numeros:
        valor = int(''.join(numeros))
        # Si el valor es muy pequeño, puede estar en millones
        if valor < 10000:
            valor = valor * 1000
        return valor

    return None

def analizar_salarios(df):
    """
    Analiza distribución de salarios
    """
    if 'salario' not in df.columns:
        return {}

    # Extraer valores numéricos
    df['salario_valor'] = df['salario'].apply(extraer_valor_salario)

    # Filtrar valores válidos
    salarios_validos = df[df['salario_valor'].notna() & (df['salario_valor'] > 1000000)]

    if len(salarios_validos) == 0:
        return {'mensaje': 'No hay datos de salarios válidos'}

    analisis_sal = {
        'total_ofertas_con_salario': len(salarios_validos),
        'salario_minimo': int(salarios_validos['salario_valor'].min()),
        'salario_maximo': int(salarios_validos['salario_valor'].max()),
        'salario_promedio': int(salarios_validos['salario_valor'].mean()),
        'salario_mediana': int(salarios_validos['salario_valor'].median()),
        'rangos_salariales': {}
    }

    # Definir rangos salariales (en SMMLV 2025: ~$1,423,500)
    smmlv = 1423500
    bins = [0, smmlv, 2*smmlv, 3*smmlv, 5*smmlv, float('inf')]
    labels = ['<1 SMMLV', '1-2 SMMLV', '2-3 SMMLV', '3-5 SMMLV', '>5 SMMLV']

    salarios_validos['rango'] = pd.cut(salarios_validos['salario_valor'], bins=bins, labels=labels)

    rangos = salarios_validos['rango'].value_counts()
    analisis_sal['rangos_salariales'] = {
        str(k): int(v) for k, v in rangos.items()
    }

    # Salarios por ocupación
    if 'cargo' in salarios_validos.columns:
        sal_por_cargo = salarios_validos.groupby('cargo')['salario_valor'].agg(['mean', 'count'])
        sal_por_cargo = sal_por_cargo[sal_por_cargo['count'] >= 2].sort_values('mean', ascending=False)

        analisis_sal['salarios_por_ocupacion'] = {
            str(k): {
                'salario_promedio': int(v['mean']),
                'num_ofertas': int(v['count'])
            } for k, v in sal_por_cargo.head(10).iterrows()
        }

    return analisis_sal

# Ejecutar análisis de salarios
print("\n=== ANÁLISIS DE SALARIOS ===")
analisis_salarios_result = analizar_salarios(df_final)

if 'mensaje' not in analisis_salarios_result:
    print(f"Ofertas con información salarial: {analisis_salarios_result['total_ofertas_con_salario']}")
    print(f"Salario promedio: ${analisis_salarios_result['salario_promedio']:,}")
    print(f"Salario mediana: ${analisis_salarios_result['salario_mediana']:,}")
    print(f"Rango: ${analisis_salarios_result['salario_minimo']:,} - ${analisis_salarios_result['salario_maximo']:,}")

    print("\n=== DISTRIBUCIÓN POR RANGOS ===")
    for rango, count in analisis_salarios_result['rangos_salariales'].items():
        print(f"{rango}: {count} ofertas")
else:
    print(analisis_salarios_result['mensaje'])

# Agregar al análisis principal
resultados_analisis['analisis_salarios'] = analisis_salarios_result

# **Exportar resultados a JSON*

In [ ]:
import json
import numpy as np
import pandas as pd
from datetime import datetime, date

def preparar_para_json(obj):
    """
    Convierte cualquier objeto (NumPy, Pandas, datetime, etc.)
    a un tipo compatible con JSON.
    """
    # Tipos numéricos
    if isinstance(obj, (np.integer, int)):
        return int(obj)
    elif isinstance(obj, (np.floating, float)):
        return float(obj)

    # Fechas y tiempos
    elif isinstance(obj, (datetime, date, pd.Timestamp, np.datetime64)):
        return str(pd.to_datetime(obj))

    # Valores faltantes
    elif isinstance(obj, (list, tuple, set, np.ndarray)):
        return [preparar_para_json(x) for x in obj]
    elif isinstance(obj, dict):
        return {str(k): preparar_para_json(v) for k, v in obj.items()}

    # Solo aplicar pd.isna a escalares, no arrays
    elif isinstance(obj, (str, bool)) or obj is None:
        return obj
    elif np.isscalar(obj):
        if pd.isna(obj):
            return None
        else:
            return obj

    return str(obj)  # fallback general



# =============================================
# EXPORTACIÓN DE RESULTADOS
# =============================================

print("\n=== EXPORTANDO RESULTADOS ===")

# Preparar dataset final para JSON
df_export = df_final.copy()

# Convertir tipos de datos en el DataFrame
for col in df_export.columns:
    df_export[col] = df_export[col].apply(preparar_para_json)

# Crear estructura JSON final
resultado_final = {
    'metadata': {
        'fecha_analisis': datetime.now().isoformat(),
        'total_mensajes_originales': int(len(df_mensajes)),
        'total_mensajes_procesados': int(len(df_final)),
        'fuente': 'Canal WhatsApp - Empleo en Bogotá'
    },
    'analisis_descriptivo': preparar_para_json(resultados_analisis),
    'ofertas_detalladas': df_export.to_dict(orient='records')
}

# Guardar JSON
with open('analisis_ofertas_empleo.json', 'w', encoding='utf-8') as f:
    json.dump(preparar_para_json(resultado_final), f, ensure_ascii=False, indent=2)

print("✓ Archivo JSON guardado: analisis_ofertas_empleo.json")

# Guardar Excel
df_export.to_excel('ofertas_procesadas.xlsx', index=False, engine='openpyxl')
print("✓ Archivo Excel guardado: ofertas_procesadas.xlsx")

# Guardar resumen CSV
df_resumen = df_final[[
    'cargo', 'empresa', 'ubicacion', 'salario',
    'nivel_educativo', 'experiencia', 'tipo_contrato',
    'codigo_cuoc', 'ocupacion_cuoc', 'grupo_cuoc'
]].copy()

df_resumen.to_csv('resumen_ofertas.csv', index=False, encoding='utf-8-sig')
print("✓ Archivo CSV guardado: resumen_ofertas.csv")


# **Visualización de estadísticas**

In [ ]:
def generar_estadisticas_visuales(resultados):
    """
    Genera un reporte en texto con las estadísticas principales
    """
    print("\n" + "="*60)
    print("REPORTE FINAL DE ANÁLISIS DE OFERTAS LABORALES")
    print("="*60)

    # Resumen general
    print("\n📊 RESUMEN GENERAL")
    print("-" * 60)
    rg = resultados['analisis_descriptivo']['resumen_general']
    print(f"Total de ofertas analizadas: {rg['total_ofertas']}")
    print(f"Ofertas con cargo identificado: {rg['ofertas_con_cargo_identificado']}")
    print(f"Ofertas clasificadas con CUOC: {rg['ofertas_con_cuoc']}")
    print(f"Total de vacantes: {rg.get('total_vacantes', 'N/A')}")
    print(f"Empresas únicas: {rg.get('empresas_unicas', 'N/A')}")

    # Top ocupaciones
    print("\n🎯 TOP 10 OCUPACIONES MÁS DEMANDADAS")
    print("-" * 60)
    for i, (cargo, freq) in enumerate(
        list(resultados['analisis_descriptivo']['ocupaciones_frecuentes'].items())[:10], 1
    ):
        print(f"{i:2d}. {cargo[:50]:<50} | {freq:>3} ofertas")

    # Grupos ocupacionales
    if resultados['analisis_descriptivo']['grupos_ocupacionales']:
        print("\n👥 DISTRIBUCIÓN POR GRUPOS OCUPACIONALES (CUOC)")
        print("-" * 60)
        for grupo, count in resultados['analisis_descriptivo']['grupos_ocupacionales'].items():
            print(f"{grupo:<30} | {count:>3} ofertas")

    # Ubicaciones
    print("\n📍 DISTRIBUCIÓN GEOGRÁFICA")
    print("-" * 60)
    for ciudad, count in resultados['analisis_descriptivo']['ubicaciones'].items():
        print(f"{ciudad:<25} | {count:>3} ofertas")

    # Nivel educativo
    print("\n🎓 REQUERIMIENTOS EDUCATIVOS")
    print("-" * 60)
    for nivel, count in resultados['analisis_descriptivo']['nivel_educativo'].items():
        print(f"{nivel:<25} | {count:>3} ofertas")

    # Salarios
    if 'analisis_salarios' in resultados['analisis_descriptivo']:
        sal = resultados['analisis_descriptivo']['analisis_salarios']
        if 'salario_promedio' in sal:
            print("\n💰 ANÁLISIS SALARIAL")
            print("-" * 60)
            print(f"Salario promedio: ${sal['salario_promedio']:,}")
            print(f"Salario mediana:  ${sal['salario_mediana']:,}")
            print(f"Rango salarial:   ${sal['salario_minimo']:,} - ${sal['salario_maximo']:,}")

            print("\nDistribución por rangos:")
            for rango, count in sal['rangos_salariales'].items():
                print(f"  {rango:<15} | {count:>3} ofertas")

    print("\n" + "="*60)
    print("Fin del reporte")
    print("="*60 + "\n")

# Generar reporte visual
generar_estadisticas_visuales(resultado_final)

# **Función completa integrada**

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, date

# --- Función para convertir datos a tipos compatibles con JSON ---
def preparar_para_json(obj):
    """
    Convierte cualquier objeto (NumPy, Pandas, datetime, etc.)
    a un tipo compatible con JSON.
    """
    if isinstance(obj, (np.integer, int)):
        return int(obj)
    elif isinstance(obj, (np.floating, float)):
        return float(obj)
    elif isinstance(obj, (datetime, date, pd.Timestamp, np.datetime64)):
        return str(pd.to_datetime(obj))
    elif isinstance(obj, (list, tuple, set, np.ndarray)):
        return [preparar_para_json(x) for x in obj]
    elif isinstance(obj, dict):
        return {str(k): preparar_para_json(v) for k, v in obj.items()}
    elif isinstance(obj, (str, bool)) or obj is None:
        return obj
    elif np.isscalar(obj):
        if pd.isna(obj):
            return None
        else:
            return obj
    return str(obj)

# --- Pipeline completo ---
def pipeline_completo(ruta_csv, ruta_cuoc_estructura=None, ruta_cuoc_indice=None):
    """
    Función que ejecuta todo el pipeline de análisis
    """
    print("="*60)
    print("INICIANDO PIPELINE DE ANÁLISIS DE OFERTAS LABORALES")
    print("="*60)

    # 1. Cargar datos
    print("\n[1/9] Cargando datos...")
    df = pd.read_csv(ruta_csv)

    # 2. Limpiar textos
    print("[2/9] Limpiando textos...")
    df['contenido_limpio'] = df['contenido'].apply(limpiar_texto)
    df = eliminar_duplicados(df)
    df = df[df['contenido_limpio'].str.len() > 50].copy()

    # 3. Extraer información
    print("[3/9] Extrayendo información...")
    info_extraida = df['contenido_limpio'].apply(extraer_informacion_oferta)
    df_info = pd.DataFrame(info_extraida.tolist())
    df_procesado = pd.concat([df.reset_index(drop=True), df_info], axis=1)

    # 4. Cargar CUOC
    print("[4/9] Cargando bases CUOC...")
    df_cuoc = None
    if ruta_cuoc_indice:
        try:
            df_cuoc = pd.read_excel(ruta_cuoc_indice)
        except Exception as e:
            print(f"  ⚠ No se pudo cargar CUOC: {e}")

    # 5. Asignar códigos CUOC
    print("[5/9] Asignando códigos CUOC...")
    if df_cuoc is not None:
        cuoc_asignado = df_procesado['cargo'].apply(
            lambda x: buscar_codigo_cuoc(x, df_cuoc, umbral=60)
        )
        df_cuoc_result = pd.DataFrame(cuoc_asignado.tolist())
        df_final = pd.concat([df_procesado.reset_index(drop=True), df_cuoc_result], axis=1)
    else:
        df_final = df_procesado.copy()

    # 6. Análisis descriptivo
    print("[6/9] Generando análisis descriptivo...")
    analisis = analisis_descriptivo(df_final)

    # 7. Análisis de salarios
    print("[7/9] Analizando salarios...")
    analisis['analisis_salarios'] = analizar_salarios(df_final)

    # 8. Preparar JSON
    print("[8/9] Preparando resultados...")
    resultado_json = {
        'metadata': {
            'fecha_analisis': datetime.now().isoformat(),
            'total_mensajes_originales': len(df),
            'total_mensajes_procesados': len(df_final)
        },
        'analisis_descriptivo': analisis,
        'ofertas_detalladas': df_final.to_dict(orient='records')
    }

    # 9. Exportar
    print("[9/9] Exportando resultados...")
    with open('analisis_ofertas_empleo.json', 'w', encoding='utf-8') as f:
        json.dump(preparar_para_json(resultado_json), f, ensure_ascii=False, indent=2)

    df_final.to_excel('ofertas_procesadas.xlsx', index=False)

    print("\n✓ Pipeline completado exitosamente")
    generar_estadisticas_visuales(resultado_json)

    return df_final, resultado_json

# === EJECUTAR PIPELINE COMPLETO ===
df_resultado, json_resultado = pipeline_completo(
    ruta_csv='mensajes.csv',
    ruta_cuoc_indice='CUOC_Indice_2024.xlsx'
)
